# T1.2 - Solution B
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes usados
import pandas as pd # Para pegar os dados dos arquivos
from IPython.display import display # Para mostrar mais de uma informação em uma mesma célula
import ast # Para transformar string/object em estruturas de dados (listas, dicionários, ...)
import numpy as np # Para obter o total de valores por um atributo
from sklearn import linear_model,datasets,svm,tree,neural_network
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.feature_selection import VarianceThreshold,SelectKBest,chi2
import matplotlib.pyplot as plt
import seaborn as sns
import math

import random
random.seed(1001001)

#colocar:

#http://scikit-learn.org/stable/modules/feature_selection.html

In [ ]:
# Carregando dados de cada um dos arquivos
data = pd.read_csv('in/winequality-red.csv',sep=';')
data.describe()

In [ ]:
#sns.set(style="ticks")
#sns.pairplot(data,hue="quality")
#plt.show()

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(data.corr(),vmax=1,square=True,annot=True,cmap='Blues')
plt.show()

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
X = data[features]
Y = data['quality']
#sns.pairplot(data,x_vars=features,y_vars='quality',kind='reg',size=7,aspect=0.5)
#plt.show()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=random.randrange(0,100),test_size=0.25)

In [ ]:
def present(x,y,h):
    fig = plt.figure(figsize = (15,15))
    size = len(data.columns)
    for i in range(0,size-1):
        ax = fig.add_subplot(math.ceil(math.sqrt(size-1)), math.ceil(math.sqrt(size-1)), i+1)
        ax.scatter(x.iloc[:,i], y,  color='red')
        ax.scatter(x.iloc[:,i], h, color='blue', linewidth=3)
    plt.show()
    #sns.pairplot(df,x_vars=features,y_vars='quality',kind='reg',size=7,aspect=0.5)
    #plt.show()

In [ ]:
def test(regr,X_train,Y_train,X_test,Y_test,linear=False):
    regr.fit(X_train, Y_train)
    Y_pred = regr.predict(X_test)
    if linear:
        print('Coefficients: \n', regr.coef_)
    print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))
    print('Variance score: %.2f' % r2_score(Y_test, Y_pred))
    return Y_pred,mean_squared_error(Y_test, Y_pred),r2_score(Y_test, Y_pred)

In [ ]:
methods = [
        linear_model.LinearRegression(),
        svm.SVR(),
        linear_model.SGDRegressor(max_iter=1000,tol=1e-3,loss="huber"),
        NearestCentroid(),
        tree.DecisionTreeRegressor(),
        neural_network.MLPRegressor(solver='lbfgs')
    ]
def test_all(X_train,Y_train,X_test,Y_test):
    errors = []
    scores = []
    for method in methods:
        print(method.__class__.__name__)
        print()
        Y_pred,error,score = test(method,X_train,Y_train,X_test,Y_test)
        print()
        scores = scores + [score]
        errors = errors + [error]
    return errors,scores

In [ ]:
Y_pred,_,_ = test(linear_model.LinearRegression(),X_train,Y_train,X_test,Y_test,True)
present(X_test,Y_test,Y_pred)

In [ ]:
Y_pred,_,_ = test(svm.SVR(),X_train,Y_train,X_test,Y_test)
present(X_test,Y_test,Y_pred)

In [ ]:
Y_pred,_,_ = test(linear_model.SGDRegressor(max_iter=1000,tol=1e-3),X_train,Y_train,X_test,Y_test)
Y_pred,_,_ = test(linear_model.SGDRegressor(max_iter=1000,tol=1e-3,loss="epsilon_insensitive"),X_train,Y_train,X_test,Y_test)
Y_pred,_,_ = test(linear_model.SGDRegressor(max_iter=1000,tol=1e-3,loss="huber"),X_train,Y_train,X_test,Y_test)
present(X_test,Y_test,Y_pred)

In [ ]:
Y_pred,_,_ = test(NearestCentroid(),X_train,Y_train,X_test,Y_test)
present(X_test,Y_test,Y_pred)

In [ ]:
Y_pred,_,_ = test(tree.DecisionTreeRegressor(),X_train,Y_train,X_test,Y_test)
present(X_test,Y_test,Y_pred)

In [ ]:
Y_pred,_,_ = test(neural_network.MLPRegressor(solver='lbfgs'),X_train,Y_train,X_test,Y_test)
present(X_test,Y_test,Y_pred)

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
X = data[features]
Y = data['quality']
#sns.pairplot(data,x_vars=features,y_vars='quality',kind='reg',size=7,aspect=0.5)
#plt.show()

In [ ]:
X = data[features]
print(np.shape(X))
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X1 = sel.fit_transform(X)
print(np.shape(X1))
X2 = SelectKBest(chi2, k=2).fit_transform(X, Y)
print(np.shape(X2))
X3 = SelectKBest(chi2, k=5).fit_transform(X, Y)
print(np.shape(X3))
#sns.pairplot(data,x_vars=features,y_vars='quality',kind='reg',size=7,aspect=0.5)
#plt.show()

In [ ]:
rand = random.randrange(0,100)
X1_train,X1_test,Y1_train,Y1_test=train_test_split(X1,Y,random_state=rand,test_size=0.25)
X2_train,X2_test,Y2_train,Y2_test=train_test_split(X2,Y,random_state=rand,test_size=0.25)
X3_train,X3_test,Y3_train,Y3_test=train_test_split(X3,Y,random_state=rand,test_size=0.25)

In [ ]:
error0,score0 = test_all(X_train,Y_train,X_test,Y_test)

In [ ]:
error1,score1 = test_all(X1_train,Y1_train,X1_test,Y1_test)

In [ ]:
error2,score2 = test_all(X2_train,Y2_train,X2_test,Y2_test)

In [ ]:
error3,score3 = test_all(X3_train,Y3_train,X3_test,Y3_test)

In [ ]:
tabelascores = pd.DataFrame(data={'no filtering':score0,'variance (80%)':score1,'chi2 k = 2':score2,'chi2 k =5':score3},
                      index=[method.__class__.__name__ for method in methods])


def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]


tabelascores.style.\
    applymap(color_negative_red).\
    apply(highlight_max)

In [ ]:
tabelaerros = pd.DataFrame(data={'no filtering':error0,'variance (80%)':error1,'chi2 k = 2':error2,'chi2 k =5':error3},
                      index=[method.__class__.__name__ for method in methods])


def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]


tabelascores.style.\
    applymap(color_negative_red).\
    apply(highlight_max)